In [5]:
import requests
import json
import time
import os
import math
from datetime import datetime, timedelta

from tqdm import tqdm

"""Функция, которая записывает в vacancies полученные данные по запросу к API "Работа России"""

def getDay(modifiedFrom: datetime):

    modifiedTo = modifiedFrom + timedelta(hours=1)
    params = {
        'modifiedFrom': f"{modifiedFrom.strftime('%Y-%m-%d')}T{modifiedFrom.hour}:00:00Z",
        'modifiedTo': f"{modifiedTo.strftime('%Y-%m-%d')}T{modifiedTo.hour}:00:00Z",
        'limit': 100,
        'offset': 0
    }

    req = requests.get('http://opendata.trudvsem.ru/api/v1/vacancies', params)
    data = json.loads(req.content.decode())
    req.close()
    if data['results']:
        vacancies = data['results']['vacancies']
    else:
        vacancies = []
    total = int(data['meta']['total'])
    if total > 10000:
        raise Exception('total more than 10000')
    for _ in range(math.ceil(total / 100)):
        time.sleep(0.1)
        params['offset'] += 1
        req = requests.get('http://opendata.trudvsem.ru/api/v1/vacancies', params)
        data = json.loads(req.content.decode())
        req.close()
        if data['results']:
            vacancies += data['results']['vacancies']
    return vacancies

# cобираем данные с 10 марта 2022 года (по дате раньше API не выдает данные),
# итерируясь по 1 часу из-за ограничения на количество данных по 10000 записей за запрос

start_time = datetime(year=2022, month=5, day=5)
end_time = datetime(year=2022, month=6, day=4)
vacancies = []

while start_time < end_time:
    try:
        vacancies += getDay(start_time)
    except Exception as e:
        print(e)
        with open('backup.json', 'w') as file:
            json.dump(vacancies, file)
            break
    start_time += timedelta(hours=1)
    print(start_time)

2022-05-05 01:00:00
2022-05-05 02:00:00
2022-05-05 03:00:00
2022-05-05 04:00:00
2022-05-05 05:00:00
2022-05-05 06:00:00
2022-05-05 07:00:00
2022-05-05 08:00:00
2022-05-05 09:00:00
2022-05-05 10:00:00
2022-05-05 11:00:00
2022-05-05 12:00:00
2022-05-05 13:00:00
2022-05-05 14:00:00
2022-05-05 15:00:00
2022-05-05 16:00:00
2022-05-05 17:00:00
2022-05-05 18:00:00
2022-05-05 19:00:00
2022-05-05 20:00:00
2022-05-05 21:00:00
2022-05-05 22:00:00
2022-05-05 23:00:00
2022-05-06 00:00:00
2022-05-06 01:00:00
2022-05-06 02:00:00
2022-05-06 03:00:00
2022-05-06 04:00:00
2022-05-06 05:00:00
2022-05-06 06:00:00
2022-05-06 07:00:00
2022-05-06 08:00:00
2022-05-06 09:00:00
2022-05-06 10:00:00
2022-05-06 11:00:00
2022-05-06 12:00:00
2022-05-06 13:00:00
2022-05-06 14:00:00
2022-05-06 15:00:00
2022-05-06 16:00:00
2022-05-06 17:00:00
2022-05-06 18:00:00
2022-05-06 19:00:00
2022-05-06 20:00:00
2022-05-06 21:00:00
2022-05-06 22:00:00
2022-05-06 23:00:00
2022-05-07 00:00:00
2022-05-07 01:00:00
2022-05-07 02:00:00


In [6]:
with open('RR_all_data_month.json', 'w') as file:
    json.dump(vacancies, file)

In [7]:
"""Преобразуем json-файл в датафрейм"""
import pandas as pd
import json
from datetime import datetime, timedelta

with open('RR_all_data_month.json', 'r') as file:
    df = pd.json_normalize(json.load(file))

In [8]:
df["vacancy.category.specialisation"].value_counts()

Производство                                                        88544
Здравоохранение и социальное обеспечение                            56458
Образование, наука                                                  50110
Продажи, закупки, снабжение, торговля                               28595
Транспорт, автобизнес, логистика, склад, ВЭД                        20482
ЖКХ, эксплуатация                                                   19499
Строительство, ремонт, стройматериалы, недвижимость                 17723
Пищевая промышленность                                              15488
Работы, не требующие квалификации                                   14084
Сельское хозяйство, экология, ветеринария                           13622
Рабочие специальности                                               10214
Консалтинг, стратегическое развитие, управление                      9537
Электроэнергетика                                                    8497
Информационные технологии, телекоммуни

In [9]:
"""Выбираем нужные специализации"""

df_filtered = df[(df["vacancy.category.specialisation"] == "Информационные технологии, телекоммуникации, связь") 
| (df["vacancy.category.specialisation"] == "Искусство, культура и развлечения") | 
(df["vacancy.category.specialisation"] == "Безопасность, службы охраны") | 
(df["vacancy.category.specialisation"] == "Государственная служба, некоммерческие организации")]
df_filtered

,vacancy.id,vacancy.source,vacancy.region.region_code,vacancy.region.name,vacancy.company.ogrn,vacancy.company.companycode,vacancy.company.hr-agency,vacancy.company.phone,vacancy.company.inn,vacancy.company.name,...,vacancy.term.text,vacancy.currency,vacancy.company.kpp,vacancy.company.email,vacancy.company.site,vacancy.company.logo,vacancy.social_protected,vacancy.company.fax,vacancy.company.code_industry_branch,vacancy.category.industry
47,d9bf4de2-8f33-11ec-8f42-bf2cfe8c828d,Служба занятости,3100000000000,Белгородская область,1023101664519,1023101664519,False,+7(915)5228531,3123035312,"ФГАОУ ВО ""БЕЛГОРОДСКИЙ ГОСУДАРСТВЕННЫЙ НАЦИОНА...",...,Социальный пакет,«руб.»,312301001,chinenova@bsu.edu.ru,https://www.bsu.edu.ru,NaN,NaN,+7(4722)301001,NaN,NaN
75,9c445c72-7f80-11ec-9bf6-bf2cfe8c828d,Служба занятости,7700000000000,г. Москва,1037700150201,1037700150201,False,+7(917)4877484,7718126549,"Федеральное казенное учреждение ""Центр инженер...",...,Социальный пакет,«руб.»,774301001,itosm@mail.ru,https://77.fsin.gov.ru/structure/citov/,NaN,NaN,NaN,NaN,NaN
93,4e7256c2-a1ed-11ec-bf37-bf2cfe8c828d,Служба занятости,7700000000000,г. Москва,1187746589359,1187746589359,False,+7(495)4190486 доб. 101,9701112870,"Государственное бюджетное учреждение ""МОСТРАНС...",...,Социальный пакетОфис в шаговой доступности от ...,«руб.»,770101001,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,f2e2bb02-8a1e-11ec-9bf6-bf2cfe8c828d,Служба занятости,3100000000000,Белгородская область,1033100503930,6cb67120-a991-11eb-aaa4-69632329477c,False,NaN,3102017305,"МДОУ ""Центр развития ребенка - детский сад №4 ...",...,Социальный пакетПолный рабочий день,«руб.»,310201001,NaN,NaN,NaN,NaN,NaN,NaN,NaN
266,dbac6b12-79ee-11ec-b378-bf2cfe8c828d,Служба занятости,7700000000000,г. Москва,2177747703098,2177747703098,False,+7(495)5056169,7722319952,ФИЛИАЛ ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНО...,...,"Социальный пакетПолная занятость, полный день.",«руб.»,775143001,zhabina_es@moslesproekt.ru,http://moslesproekt.ru/contacts.html,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445348,ebe97a00-9cc5-11ec-add7-bf2cfe8c828d,Служба занятости,2300000000000,Краснодарский край,1152315001838,1152315001838,False,+7(988)1367080,2315982280,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""АЛУ...",...,Социальный пакет,«руб.»,231501001,buh@alumann.ru,http://zakaz@alumann.ru,NaN,NaN,NaN,NaN,NaN
445361,68daf982-7264-11ec-b378-bf2cfe8c828d,Служба занятости,2300000000000,Краснодарский край,1152308008050,1152308008050,False,+7(861)2592042 доб. 226,2308222063,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""МУНИ...",...,Социальный пакетВ творческом коллективе,«руб.»,230801001,kadry@tvkrasnodar.ru,NaN,NaN,Инвалиды,NaN,NaN,NaN
445378,2bc5d030-9c12-11ea-94f4-bf2cfe8c828d,Служба занятости,2300000000000,Краснодарский край,1072333000442,1072333000442,False,+7(918)0404122,2333012214,"МУНИЦИПАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ""БИБЛИОТЕЧНА...",...,Социальный пакет,«руб.»,233301001,griv.biblioteca.93@mail.ru,NaN,NaN,NaN,NaN,NaN,NaN
445379,6d50c452-2034-11eb-94f4-bf2cfe8c828d,Служба занятости,2300000000000,Краснодарский край,1072333000442,1072333000442,False,+7(918)0404122,2333012214,"МУНИЦИПАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ""БИБЛИОТЕЧНА...",...,Социальный пакет,«руб.»,233301001,griv.biblioteca.93@mail.ru,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_filtered["vacancy.category.specialisation"].value_counts()

Информационные технологии, телекоммуникации, связь    8127
Государственная служба, некоммерческие организации    7358
Искусство, культура и развлечения                     6801
Безопасность, службы охраны                           4911
Name: vacancy.category.specialisation, dtype: int64

In [15]:
df_filtered

,vacancy.id,vacancy.source,vacancy.region.region_code,vacancy.region.name,vacancy.company.ogrn,vacancy.company.companycode,vacancy.company.hr-agency,vacancy.company.phone,vacancy.company.inn,vacancy.company.name,...,vacancy.term.text,vacancy.currency,vacancy.company.kpp,vacancy.company.email,vacancy.company.site,vacancy.company.logo,vacancy.social_protected,vacancy.company.fax,vacancy.company.code_industry_branch,vacancy.category.industry
47,d9bf4de2-8f33-11ec-8f42-bf2cfe8c828d,Служба занятости,3100000000000,Белгородская область,1023101664519,1023101664519,False,+7(915)5228531,3123035312,"ФГАОУ ВО ""БЕЛГОРОДСКИЙ ГОСУДАРСТВЕННЫЙ НАЦИОНА...",...,Социальный пакет,«руб.»,312301001,chinenova@bsu.edu.ru,https://www.bsu.edu.ru,NaN,NaN,+7(4722)301001,NaN,NaN
75,9c445c72-7f80-11ec-9bf6-bf2cfe8c828d,Служба занятости,7700000000000,г. Москва,1037700150201,1037700150201,False,+7(917)4877484,7718126549,"Федеральное казенное учреждение ""Центр инженер...",...,Социальный пакет,«руб.»,774301001,itosm@mail.ru,https://77.fsin.gov.ru/structure/citov/,NaN,NaN,NaN,NaN,NaN
93,4e7256c2-a1ed-11ec-bf37-bf2cfe8c828d,Служба занятости,7700000000000,г. Москва,1187746589359,1187746589359,False,+7(495)4190486 доб. 101,9701112870,"Государственное бюджетное учреждение ""МОСТРАНС...",...,Социальный пакетОфис в шаговой доступности от ...,«руб.»,770101001,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,f2e2bb02-8a1e-11ec-9bf6-bf2cfe8c828d,Служба занятости,3100000000000,Белгородская область,1033100503930,6cb67120-a991-11eb-aaa4-69632329477c,False,NaN,3102017305,"МДОУ ""Центр развития ребенка - детский сад №4 ...",...,Социальный пакетПолный рабочий день,«руб.»,310201001,NaN,NaN,NaN,NaN,NaN,NaN,NaN
266,dbac6b12-79ee-11ec-b378-bf2cfe8c828d,Служба занятости,7700000000000,г. Москва,2177747703098,2177747703098,False,+7(495)5056169,7722319952,ФИЛИАЛ ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНО...,...,"Социальный пакетПолная занятость, полный день.",«руб.»,775143001,zhabina_es@moslesproekt.ru,http://moslesproekt.ru/contacts.html,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445348,ebe97a00-9cc5-11ec-add7-bf2cfe8c828d,Служба занятости,2300000000000,Краснодарский край,1152315001838,1152315001838,False,+7(988)1367080,2315982280,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""АЛУ...",...,Социальный пакет,«руб.»,231501001,buh@alumann.ru,http://zakaz@alumann.ru,NaN,NaN,NaN,NaN,NaN
445361,68daf982-7264-11ec-b378-bf2cfe8c828d,Служба занятости,2300000000000,Краснодарский край,1152308008050,1152308008050,False,+7(861)2592042 доб. 226,2308222063,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""МУНИ...",...,Социальный пакетВ творческом коллективе,«руб.»,230801001,kadry@tvkrasnodar.ru,NaN,NaN,Инвалиды,NaN,NaN,NaN
445378,2bc5d030-9c12-11ea-94f4-bf2cfe8c828d,Служба занятости,2300000000000,Краснодарский край,1072333000442,1072333000442,False,+7(918)0404122,2333012214,"МУНИЦИПАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ""БИБЛИОТЕЧНА...",...,Социальный пакет,«руб.»,233301001,griv.biblioteca.93@mail.ru,NaN,NaN,NaN,NaN,NaN,NaN
445379,6d50c452-2034-11eb-94f4-bf2cfe8c828d,Служба занятости,2300000000000,Краснодарский край,1072333000442,1072333000442,False,+7(918)0404122,2333012214,"МУНИЦИПАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ""БИБЛИОТЕЧНА...",...,Социальный пакет,«руб.»,233301001,griv.biblioteca.93@mail.ru,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df_filtered.to_csv('RR_4_specialisations_month.json.csv')

In [11]:
"""Преобразуем время в формат datetime"""

def change_date(x):
    try:
        date = datetime.strptime(x, "%Y-%m-%d")
        
    except Exception as e: # нам известно о двух ошибках в записи времени
         # если не получилось преобразовать в datetime, то исправляем эти две ошибки 
        if x == "333322-03-11":
            return datetime.strptime('2022-03-11', "%Y-%m-%d")
        return datetime.strptime('2022-03-28', "%Y-%m-%d")
        
    return date
    
df_filtered["vacancy.creation-date"] = df_filtered["vacancy.creation-date"].apply(change_date)

/var/folders/pf/sj5w0_zs1fs2qhqbvvcnn2500000gn/T/ipykernel_1424/149301025.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["vacancy.creation-date"] = df_filtered["vacancy.creation-date"].apply(change_date)


In [12]:
"""Валидируем корректность даты"""

def clean_date(x):
    if x > datetime.strptime('2022-06-05', "%Y-%m-%d"):
        return None
    elif x < datetime.strptime('2022-05-04', "%Y-%m-%d"):
        return None
    return x

df_filtered["vacancy.creation-date"] = df_filtered["vacancy.creation-date"].apply(clean_date)

/var/folders/pf/sj5w0_zs1fs2qhqbvvcnn2500000gn/T/ipykernel_1424/1922441678.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["vacancy.creation-date"] = df_filtered["vacancy.creation-date"].apply(clean_date)


In [13]:
first_date = datetime.strptime('2022-05-04', "%Y-%m-%d")
end_date = datetime.strptime('2022-06-05', "%Y-%m-%d")
df_result = df_filtered.loc[(df_filtered["vacancy.creation-date"] < end_date) & (df_filtered["vacancy.creation-date"] > first_date)]
df_result = df_result.reset_index().drop(columns = ['index'])

In [14]:
df_result

,vacancy.id,vacancy.source,vacancy.region.region_code,vacancy.region.name,vacancy.company.ogrn,vacancy.company.companycode,vacancy.company.hr-agency,vacancy.company.phone,vacancy.company.inn,vacancy.company.name,...,vacancy.term.text,vacancy.currency,vacancy.company.kpp,vacancy.company.email,vacancy.company.site,vacancy.company.logo,vacancy.social_protected,vacancy.company.fax,vacancy.company.code_industry_branch,vacancy.category.industry
0,1ce307a5-cccf-11ec-8848-813aa7e4e138,Работодатель,2500000000000,Приморский край,1022502287345,1022502287345,False,+7(423)2317620,2540019578,"ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ""ВОЕННЫЙ КОМИС...",...,NaN,«руб.»,254001002,marusya260961@mail.ru,NaN,NaN,NaN,+7(423)2346529,NaN,NaN
1,d63a0a65-ccce-11ec-bd91-5b1b13d7f12f,Работодатель,2500000000000,Приморский край,1022502287345,1022502287345,False,+7(423)2317620,2540019578,"ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ""ВОЕННЫЙ КОМИС...",...,NaN,«руб.»,254001002,marusya260961@mail.ru,NaN,NaN,NaN,+7(423)2346529,NaN,NaN
2,4e9d1755-ccce-11ec-9011-4febb26dc4ec,Работодатель,2500000000000,Приморский край,1022502287345,1022502287345,False,+7(423)2317620,2540019578,"ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ""ВОЕННЫЙ КОМИС...",...,NaN,«руб.»,254001002,marusya260961@mail.ru,NaN,NaN,NaN,+7(423)2346529,NaN,NaN
3,ca9d8bc5-cccc-11ec-9011-4febb26dc4ec,Работодатель,2500000000000,Приморский край,1022502287345,1022502287345,False,+7(423)2317620,2540019578,"ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ""ВОЕННЫЙ КОМИС...",...,NaN,«руб.»,254001002,marusya260961@mail.ru,NaN,NaN,NaN,+7(423)2346529,NaN,NaN
4,759e3fb5-cccd-11ec-9011-4febb26dc4ec,Работодатель,2500000000000,Приморский край,1022502287345,1022502287345,False,+7(423)2317620,2540019578,"ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ""ВОЕННЫЙ КОМИС...",...,NaN,«руб.»,254001002,marusya260961@mail.ru,NaN,NaN,NaN,+7(423)2346529,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7143,00444f72-cd4f-11ec-9aad-bf2cfe8c828d,Служба занятости,7700000000000,г. Москва,1037700067613,1037700067613,False,+7(926)5233271,7704233220,"Общество с ограниченной ответственностью ""Комп...",...,Дополнительное медицинское страхование,«руб.»,770401001,rebeka9@yandex.ru,https://genatsvale.rest/,NaN,NaN,NaN,NaN,NaN
7144,abe1fc22-cd4e-11ec-9aad-bf2cfe8c828d,Служба занятости,7700000000000,г. Москва,1187746557360,1187746557360,False,+7(985)2242731,7720431892,"Общество с ограниченной ответственностью ""В.М""",...,ЗдравохранениеПредоставление жильянормированны...,«руб.»,772001001,Interseil@mail.ru,NaN,NaN,NaN,NaN,NaN,NaN
7145,e91b4542-e065-11ec-9aad-bf2cfe8c828d,Служба занятости,2300000000000,Краснодарский край,1222300009205,1222300009205,False,+7(918)1970001,2372029966,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ЗАВОД...,...,Социальный пакетУсловия труда на рабочем месте...,«руб.»,237201001,brekhina_tv@zskkuban.ru,NaN,NaN,NaN,NaN,NaN,NaN
7146,cb586072-d605-11ec-9aad-bf2cfe8c828d,Служба занятости,2300000000000,Краснодарский край,1022305030329,1022305030329,False,+7(86151)30555,2358005755,МБДОУ Д/С N 3 МОЩР СТ.НОВОЩЕРБИНОВСКАЯ,...,Социальный пакет,«руб.»,235801001,detskiysadv3@mail.ru,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
"""Сохраняем данные в файлы"""
# df_result.to_json('result_data.json')
df_result.to_csv('RR_4_specialisations_month.json.csv')